# Interpretabilidad del modelo de detección de comentarios tóxicos

Este análisis tiene como objetivo explicar de forma clara y visual cómo el modelo de clasificación de comentarios tóxicos toma sus decisiones.

Utilizaremos la librería **SHAP** para interpretar los resultados del modelo. SHAP nos ayuda a entender qué palabras influyen más en que un comentario sea clasificado como tóxico o no.

Este análisis está pensado para ser comprendido tanto por perfiles técnicos como no técnicos (stakeholders).


In [ ]:
import joblib
import shap
import numpy as np

# Cargamos modelo y vectorizador
model = joblib.load("../final_model/modelo_toxicidad_xgboost_final.pkl")
vectorizer = joblib.load("../final_model/vectorizer_toxicidad_final.pkl")

# Comentarios de ejemplo
comments = [
    "You're disgusting and stupid.",          # tóxico
    "Thank you for this amazing explanation.", # no tóxico
    "I don't think this was helpful at all."   # ambiguo
]

# Vectorizamos comentarios
X = vectorizer.transform(comments)

# Obtenemos los nombres reales de las features (tokens del vocabulario)
feature_names = vectorizer.get_feature_names_out(); # Usamos TreeExplainer porque el modelo es un XGBoost basado en árboles
# Convertimos la matriz TF-IDF a densa con .toarray(), que es lo que necesita SHAP
explainer = shap.TreeExplainer(model)
shap_values = explainer(X.toarray())  # X fue definido previamente como vectorizado de ejemplos; # Mostramos los tokens más influyentes a nivel global (positivo y negativo)
shap.summary_plot(shap_values, X.toarray(), feature_names=feature_names)


In [ ]:
# Creamos el explicador SHAP con nombres reales
explainer = shap.Explainer(model, feature_names=feature_names)

# Obtenemos valores SHAP para cada comentario vectorizado
shap_values = explainer(X)


## Explicabilidad global con SHAP

Vamos a utilizar SHAP para entender qué palabras del vocabulario aprendido por el modelo tienen un mayor impacto, en promedio, al predecir toxicidad.

El gráfico generado por `shap.summary_plot` muestra:

- En el eje **x**, el valor SHAP medio de cada palabra (cuánto contribuye, en general, a aumentar o disminuir la probabilidad de toxicidad).
- En el eje **y**, las palabras más influyentes (ordenadas de mayor a menor impacto).
- **Cada punto** representa una aparición de esa palabra en algún comentario.
- **El color** indica el valor del token en el comentario (alto = frecuente, bajo = poco frecuente).

Este análisis es útil para identificar qué términos el modelo considera más tóxicos en general.


In [ ]:
shap.plots.bar(shap_values)  # ✅ Ahora muestra los tokens correctamente


## Explicabilidad global – resumen por importancia media

Este gráfico de barras muestra las palabras que tienen **mayor impacto medio absoluto** sobre las predicciones del modelo. Es decir, aquellas que más influyen en general, independientemente de si ese impacto es positivo o negativo.

- El valor numérico a la derecha de cada barra representa la **media del valor SHAP** absoluto de esa palabra.
- A diferencia del gráfico anterior, **no se muestra la dispersión ni el color por valor**: este gráfico es útil para tener una visión rápida y priorizada de las palabras que más peso tienen en las decisiones del modelo.

Este análisis ayuda a identificar qué tokens tienen mayor poder predictivo a lo largo de todo el conjunto de datos.


## ⚠️ Reflexión crítica: posibles sesgos y asociaciones espurias

Uno de los tokens destacados en los gráficos SHAP es **"ferguson"**. Aunque no es una palabra ofensiva en sí misma, aparece con una contribución media notable en las predicciones de toxicidad.

Este caso es un ejemplo de lo que en aprendizaje automático se conoce como **asociación espuria**: el modelo ha aprendido que la palabra "ferguson" suele aparecer en comentarios tóxicos, no porque sea una palabra tóxica, sino por el contexto social o mediático en el que fue utilizada en el dataset original.

Esto revela un posible **sesgo en los datos de entrenamiento**. Si no se revisa, el modelo podría clasificar como tóxicos comentarios informativos o respetuosos que simplemente mencionen ciertos temas o lugares sensibles.

🔍 Este hallazgo justifica el uso de técnicas de interpretabilidad como **SHAP**, que nos permiten no solo entender cómo decide el modelo, sino también detectar y corregir errores, sesgos o asociaciones no deseadas.


In [ ]:
import matplotlib.pyplot as plt

# Comentario 1: Tóxico
plt.figure()
shap.plots.waterfall(shap_values[0], max_display=10, show=False)
plt.title("Comentario 1 – Tóxico", fontsize=14)
plt.tight_layout()
plt.show()

# Comentario 2: No tóxico
plt.figure()
shap.plots.waterfall(shap_values[1], max_display=10, show=False)
plt.title("Comentario 2 – No tóxico", fontsize=14)
plt.tight_layout()
plt.show()



## Ejemplos concretos: ¿Cómo decide el modelo en cada comentario?

Los siguientes gráficos muestran cómo el modelo llega a su decisión en **dos casos reales**. Utilizamos la visualización tipo `waterfall` de SHAP, que descompone la predicción en contribuciones individuales de cada token:

- 🔴 **Rojo**: palabras que empujan la predicción hacia **tóxico**
- 🔵 **Azul**: palabras que empujan hacia **no tóxico**
- `1·palabra` → la palabra está presente en el comentario
- `0·palabra` → la palabra no aparece, pero SHAP estima su efecto si lo hiciera

---

### 🔴 Ejemplo 1 – Comentario clasificado como **tóxico**

- La palabra **"stupid"** está presente y tiene un peso importante hacia la toxicidad.
- El modelo interpreta este término como altamente ofensivo, y su presencia basta para elevar la predicción.
- Los demás tokens tienen impacto bajo o nulo.

---

### 🔵 Ejemplo 2 – Comentario clasificado como **no tóxico**

- Aunque el texto podría parecer una crítica o estar en un contexto delicado, no contiene insultos ni lenguaje violento.
- El modelo mantiene la predicción baja y descarta la toxicidad.
- Esto sugiere que el modelo distingue correctamente entre **contenido crítico** y **contenido ofensivo**.

---

Gracias a esta explicación local, podemos confirmar que:

- El modelo no actúa como una caja negra: sus decisiones son rastreables y comprensibles.
- Penaliza el uso explícito de lenguaje ofensivo, no los temas tratados.
- SHAP nos permite validar que no hay sesgos evidentes en estos ejemplos, y que el modelo responde bien ante diferentes contextos.

Estas visualizaciones refuerzan la confianza en el comportamiento del modelo y permiten detectar posibles errores antes de su despliegue.


## Conclusiones finales y reflexión crítica

Durante este análisis, hemos evaluado no solo el rendimiento del modelo de clasificación de toxicidad, sino también su **capacidad de toma de decisiones y los posibles sesgos presentes** en su razonamiento interno, gracias a la interpretabilidad con SHAP.

A través de gráficos individuales (`waterfall`) y globales (`bar`), hemos podido identificar qué tokens tienen más peso en la toma de decisiones del modelo y cómo interactúan en diferentes tipos de comentarios (tóxicos y neutros).

---

### ¿Tiene sesgos nuestro modelo?

**Sí, muestra indicios de sesgos contextuales aprendidos durante el entrenamiento**, aunque no siempre se traducen en clasificaciones erróneas.

#### Justificación:

- Palabras sensibles como `"ferguson"`, `"cnn"`, `"black"`, `"african american"`, `"muslim"`, etc. aparecen recurrentemente como tokens que **empujan hacia la predicción de toxicidad**, tanto en gráficos individuales como globales.

- En algunos comentarios **objetivos y no ofensivos**, el modelo muestra una **probabilidad elevada de toxicidad**, lo que indica que asocia ciertas palabras a patrones aprendidos como "tóxicos", **sin tener en cuenta el tono real del comentario**.

- Sin embargo, el modelo **no clasifica automáticamente como tóxicos** todos los comentarios que contienen esos términos. Esto sugiere que:

  🔵 El modelo **no tiene un sesgo absoluto ni determinista**  
  🔴 Pero sí ha aprendido **correlaciones espurias**  
  _(Ejemplo: "ferguson" aparece en muchos comentarios tóxicos del dataset → el modelo lo aprende como indicador de toxicidad)_

---

### Conclusión crítica y valor añadido de SHAP

Este análisis muestra que el modelo, aunque funcional, **refleja los sesgos latentes de los datos de entrenamiento**. Por ello:

- **SHAP es fundamental no solo para explicar predicciones, sino también para auditar el modelo**
- Detectar tokens con impacto indebido permite reflexionar sobre la necesidad de:
  - Mejorar el dataset (más ejemplos neutrales con palabras sensibles)
  - Aplicar técnicas de balanceo o filtrado semántico
  - Incluir controles éticos antes de producción

💡 En definitiva, este análisis no solo explica el "qué" del modelo, sino también el "por qué", y **abre la puerta a una mejora consciente y responsable de la IA aplicada al lenguaje.**
